# Notebook 5: Gender Projection Analysis and Visualization

**Objective:** Analyze occupational gender bias using geometric projection onto a defined gender axis ('man' - 'woman'). This involves:
1. Loading the pre-generated GPT-2 embeddings and the validated occupation dictionary.
2. Defining the gender axis based on the vector difference between 'man' and 'woman' embeddings.
3. Normalizing the gender axis.
4. Calculating the projection score for each occupation by taking the dot product of its embedding with the normalized gender axis.
5. Visualizing the projection scores using a bar plot, highlighting the most extreme positive and potentially negative scores (similar to Figure 6).
6. Saving the calculated projection scores.

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors # For color mapping
import matplotlib.cm as cm # For colormaps
from pathlib import Path
import os

## 2. Configuration

In [2]:
# --- Paths ---
# Get project root assuming the notebook is in 'notebooks' directory
current_dir = Path.cwd()
project_root = current_dir.parent

In [3]:
# Input files
DICT_INPUT_CSV = project_root / 'results' / 'occupation_dictionary_validated.csv'
EMBEDDING_INPUT_FILE = project_root / 'results' / 'gpt2_static_embeddings.npz'

In [4]:
# Output files
RESULTS_DIR = project_root / 'results'
PROJECTION_SCORES_OUTPUT_CSV = RESULTS_DIR / 'gender_projection_results.csv'
PROJECTION_BARPLOT_OUTPUT_PNG = RESULTS_DIR / 'gender_projection_barplot_figure6.png'

In [5]:
# --- Parameters ---
GENDER_AXIS_TERMS = ('man', 'woman') # Terms defining the axis
# Number of occupations for the bar plot (e.g., 20 highest positive, 20 lowest - total 40 as per paper)
BARPLOT_NUM_OCCUPATIONS = 40

In [6]:
# Create results directory if it doesn't exist
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

## 3. Load Data

In [7]:
# Load occupation dictionary
try:
    df_dictionary = pd.read_csv(DICT_INPUT_CSV)
    print(f"Loaded dictionary with {len(df_dictionary)} occupations.")
except FileNotFoundError:
    print(f"Error: Dictionary file not found at {DICT_INPUT_CSV}")
    print("Please ensure Notebook 1 ran successfully.")
    raise
except Exception as e:
    print(f"Error loading dictionary CSV: {e}")
    raise

Loaded dictionary with 100 occupations.


In [8]:
# Load embeddings
try:
    embeddings_data = np.load(EMBEDDING_INPUT_FILE, allow_pickle=True)
    embeddings = {key: embeddings_data[key] for key in embeddings_data.files}
    print(f"Loaded {len(embeddings)} embeddings.")
except FileNotFoundError:
    print(f"Error: Embeddings file not found at {EMBEDDING_INPUT_FILE}")
    print("Please ensure Notebook 2 ran successfully.")
    raise
except Exception as e:
    print(f"Error loading embeddings file: {e}")
    raise

Loaded 104 embeddings.


In [9]:
# Verify essential gender embeddings for the axis are loaded
term1, term2 = GENDER_AXIS_TERMS
if term1 not in embeddings or term2 not in embeddings:
    missing_axis_terms = [t for t in GENDER_AXIS_TERMS if t not in embeddings]
    raise ValueError(f"Essential gender axis term embeddings missing from file: {missing_axis_terms}")

## 4. Define Gender Axis and Calculate Projections

In [10]:
# Define gender axis
man_emb = embeddings[term1]
woman_emb = embeddings[term2]
gender_axis = man_emb - woman_emb

In [11]:
# Normalize the gender axis
norm = np.linalg.norm(gender_axis)
if norm == 0:
    raise ValueError("Gender axis vector has zero magnitude. Embeddings for 'man' and 'woman' might be identical.")
gender_axis_norm = gender_axis / norm
print(f"Gender axis ('{term1}' - '{term2}') defined and normalized.")

Gender axis ('man' - 'woman') defined and normalized.


In [12]:
# Calculate projection scores
projection_results = []
processed_occupations_proj = set()

In [13]:
for index, row in df_dictionary.iterrows():
    occupation = row['occupation']

    if occupation in processed_occupations_proj:
        continue
    processed_occupations_proj.add(occupation)

    if occupation in embeddings:
        occ_emb = embeddings[occupation]
        try:
            # Calculate dot product with normalized axis
            projection_score = np.dot(occ_emb, gender_axis_norm)
            projection_results.append({
                'occupation': occupation,
                'projection_score': projection_score
            })
        except Exception as e:
            print(f"Error calculating projection for '{occupation}': {e}")
    else:
         print(f"Warning: Embedding not found for occupation '{occupation}' during projection. Skipping.")

In [14]:
# Convert results to DataFrame
df_projections = pd.DataFrame(projection_results)

In [15]:
if df_projections.empty:
     print("Error: No projection scores were generated. Check input data and embeddings.")
else:
     print(f"Calculated projection scores for {len(df_projections)} occupations.")

Calculated projection scores for 100 occupations.


## 5. Merge Results and Calculate Median

In [16]:
# Merge calculated scores back into the main dictionary dataframe
# Use outer merge first to keep all occupations, then merge projections
df_merged_proj = pd.merge(df_dictionary, df_projections, on='occupation', how='left')

In [17]:
# Calculate the median projection score across ALL occupations with a valid score
median_projection_score = df_merged_proj['projection_score'].median()
print(f"Median projection score across all occupations: {median_projection_score:.4f}")

Median projection score across all occupations: 14.3177


In [18]:
print(f"Final merged dataframe shape (including projections): {df_merged_proj.shape}")
print("Merged DataFrame sample:")
print(df_merged_proj[['occupation', 'bls_label', 'projection_score']].head())

Final merged dataframe shape (including projections): (100, 7)
Merged DataFrame sample:
            occupation           bls_label  projection_score
0      chief executive             neutral          8.981798
1              manager             neutral          7.611988
2    marketing manager             neutral          0.537213
3        sales manager             neutral         10.753222
4  fundraising manager  female-stereotyped         -2.335274


## 6. Save Calculated Scores

In [19]:
print(f"\nSaving calculated projection scores to {PROJECTION_SCORES_OUTPUT_CSV}...")
try:
    # Save the merged dataframe which now includes projection scores
    df_merged_proj.to_csv(PROJECTION_SCORES_OUTPUT_CSV, index=False, encoding='utf-8')
    print("Projection scores saved successfully.")
except Exception as e:
    print(f"Error saving scores CSV: {e}")


Saving calculated projection scores to /Users/jessie/Documents/Projects/master_thesis_llms_bias/results/gender_projection_results.csv...
Projection scores saved successfully.


## 7. Generate Projection Bar Plot (Figure 6)

In [20]:
if df_merged_proj.empty or 'projection_score' not in df_merged_proj.columns:
    print("Skipping projection bar plot generation: No data or 'projection_score' column missing.")
else:
    # Sort by projection score (descending, highest score is most 'man'-aligned)
    df_sorted_proj = df_merged_proj.sort_values('projection_score', ascending=False).dropna(subset=['projection_score'])

    # Select top N/2 and bottom N/2 (N=BARPLOT_NUM_OCCUPATIONS)
    n_each_proj = BARPLOT_NUM_OCCUPATIONS // 2
    if len(df_sorted_proj) < BARPLOT_NUM_OCCUPATIONS:
        print(f"Warning: Fewer than {BARPLOT_NUM_OCCUPATIONS} occupations with valid scores ({len(df_sorted_proj)}). Plotting all available.")
        df_selected_proj = df_sorted_proj
    elif len(df_sorted_proj) >= n_each_proj * 2:
        top_n_proj = df_sorted_proj.head(n_each_proj)
        # Important: bottom N/2 are lowest scores (least man-aligned / potentially woman-aligned)
        bottom_n_proj = df_sorted_proj.tail(n_each_proj)
        # Combine and re-sort by score for the plot order (descending)
        df_selected_proj = pd.concat([top_n_proj, bottom_n_proj]).sort_values('projection_score', ascending=False)
    else: # Handle cases with < n_each*2 but >= n_each
         print(f"Warning: Only {len(df_sorted_proj)} occupations available for bar plot. Plotting all.")
         df_selected_proj = df_sorted_proj


    if df_selected_proj.empty:
        print("Skipping projection bar plot: No occupations selected.")
    else:
        # --- Create Plot ---
        try:
            # Adjust figure height
            fig_height_proj = max(8, len(df_selected_proj) * 0.25) # Adjust multiplier for potentially many bars
            plt.figure(figsize=(12, fig_height_proj))

            # --- Color Mapping based on Score ---
            # Use a diverging colormap like 'coolwarm' or 'RdBu_r' (Red=Positive/Male, Blue=Negative/Female)
            cmap = cm.coolwarm
            # Create a normalizer: map scores to the 0-1 range for the colormap
            # Find min/max scores *within the selected data* for normalization range
            vmin = df_selected_proj['projection_score'].min()
            vmax = df_selected_proj['projection_score'].max()
            # Center the normalization around 0 if scores span positive and negative
            norm_center = 0
            norm = mcolors.TwoSlopeNorm(vcenter=norm_center, vmin=vmin, vmax=vmax)
            # Create colors for each bar
            colors = cmap(norm(df_selected_proj['projection_score'].values))
            # --- End Color Mapping ---


            ax_proj_barplot = sns.barplot(
                x='projection_score',
                y='occupation',
                data=df_selected_proj,
                palette=colors, # Pass the generated colors directly
                orient='h'
                # Note: 'hue' is not used here as color represents the score itself
            )

            plt.title(f'Gender Projection Scores (Higher = More "Man"-Associated)', fontsize=16)
            plt.xlabel('Gender Projection Score (on "man" - "woman" axis)', fontsize=12)
            plt.ylabel('Occupation', fontsize=12)

            # Add vertical line for the median score across ALL occupations
            plt.axvline(x=median_projection_score, color='black', linestyle='--', linewidth=1.5, label=f'Median = {median_projection_score:.2f}')
            plt.legend() # Show the median line label

            # Optional: Add line at x=0 for reference if it's meaningful
            # plt.axvline(x=0, color='grey', linestyle=':', linewidth=1)

            plt.tight_layout()

            # Save the plot
            plt.savefig(PROJECTION_BARPLOT_OUTPUT_PNG, dpi=300, bbox_inches='tight')
            print(f"Projection bar plot saved successfully to {PROJECTION_BARPLOT_OUTPUT_PNG}")
            plt.close() # Close the plot figure

        except Exception as e:
            print(f"Error generating projection bar plot: {e}")
            plt.close()

/var/folders/jj/84v_jpsx4rj3__y62xh3klxm0000gn/T/ipykernel_58003/612873535.py:47: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax_proj_barplot = sns.barplot(
/var/folders/jj/84v_jpsx4rj3__y62xh3klxm0000gn/T/ipykernel_58003/612873535.py:47: UserWarning: Numpy array is not a supported type for `palette`. Please convert your palette to a list. This will become an error in v0.14
  ax_proj_barplot = sns.barplot(


Projection bar plot saved successfully to /Users/jessie/Documents/Projects/master_thesis_llms_bias/results/gender_projection_barplot_figure6.png
